# EMG模型知识蒸馏训练 - 正确版本 V2

**时间序列事件检测任务的知识蒸馏**

## 目标
- Teacher: Meta官方DiscreteGesturesModule (6.5M参数)
- Student: TimeSeriesStudent (140K参数, 45x压缩)
- 输出: 时间序列 `[batch, 9, 1598]` (正确！)
- 预期准确度: 35-45% (Teacher=47%)

## 修复的关键问题
✅ 输出格式: 时间序列而非单分类  
✅ 损失函数: BCE + MSE蒸馏  
✅ 评估方法: 使用Teacher的collect_metric  
✅ 任务定义: 多标签时间序列事件检测

## 1. 环境设置与安装

In [1]:
# 检测运行环境
import os
import sys

IN_COLAB = 'google.colab' in sys.modules
print(f"运行环境: {'Google Colab' if IN_COLAB else 'Local'}")

if IN_COLAB:
    print("🔧 在Colab环境中设置...")
    # 切换到代码目录
    if not os.path.exists('/content/generic-neuromotor-interface'):
        !git clone https://github.com/facebookresearch/generic-neuromotor-interface.git
    os.chdir('/content/generic-neuromotor-interface')
    
    # 安装依赖
    !pip install -q pytorch-lightning==1.8.6 hydra-core==1.3.2 omegaconf==2.3.0 h5py
    !pip install -q -e .
    
    print("✅ 环境设置完成")
else:
    print("✅ 本地环境")

运行环境: Google Colab
🔧 在Colab环境中设置...
  Preparing metadata (setup.py) ... done
✅ 环境设置完成


In [2]:
# 挂载Google Drive（可选 - 用于保存checkpoint）
USE_GOOGLE_DRIVE = False  # 设置为True以使用Google Drive

if IN_COLAB and USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    SAVE_DIR = '/content/drive/MyDrive/emg_distillation'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f"✅ 将保存到: {SAVE_DIR}")
else:
    SAVE_DIR = './logs/distillation'
    print(f"✅ 将保存到: {SAVE_DIR}")

✅ 将保存到: ./logs/distillation


## 2. 下载数据和Teacher模型

In [3]:
# 下载EMG数据和Teacher模型
from generic_neuromotor_interface.scripts.download_data import download_data
from generic_neuromotor_interface.scripts.download_models import download_models

TASK = "discrete_gestures"
DATA_PATH = "/content/emg_data" if IN_COLAB else "~/emg_data"
MODELS_PATH = "/content/emg_models" if IN_COLAB else "~/emg_models"

# 下载数据（完整数据集）
if not os.path.exists(os.path.expanduser(DATA_PATH)):
    print("📥 下载EMG数据...")
    download_data(TASK, "full_data", DATA_PATH)
    print("✅ 数据下载完成")
else:
    print("✅ 数据已存在")

# 下载Teacher模型
if not os.path.exists(os.path.expanduser(MODELS_PATH)):
    print("📥 下载Teacher模型...")
    download_models(TASK, MODELS_PATH)
    print("✅ 模型下载完成")
else:
    print("✅ 模型已存在")

TEACHER_CHECKPOINT = os.path.join(os.path.expanduser(MODELS_PATH), TASK, "model_checkpoint.ckpt")
print(f"\n📍 Teacher checkpoint: {TEACHER_CHECKPOINT}")

📥 下载EMG数据...


Unzipping the data...


Extracting files: 100%|██████████| 100/100 [09:46<00:00,  5.87s/it]


Deleting the tar file after extraction: discrete_gestures_full_data.tar


Data for discrete_gestures (full_data) downloaded and extracted to /content/emg_data
✅ 数据下载完成
✅ 模型已存在

📍 Teacher checkpoint: /content/emg_models/discrete_gestures/model_checkpoint.ckpt


## 3. 配置超参数

In [4]:
# 蒸馏训练超参数
EXPERIMENT_ID = "distill_timeseries_v2"
MAX_EPOCHS = 30  # 快速训练，可后续增加到50-100
BATCH_SIZE = 64  # T4 GPU优化（可调整为32/128）
TEMPERATURE = 4.0  # 蒸馏温度
ALPHA = 0.7  # 蒸馏损失权重（0.7*蒸馏 + 0.3*真实标签）
LEARNING_RATE = 5e-4

print("="*60)
print("🎯 训练配置")
print("="*60)
print(f"实验ID: {EXPERIMENT_ID}")
print(f"最大轮数: {MAX_EPOCHS}")
print(f"批大小: {BATCH_SIZE}")
print(f"蒸馏温度: {TEMPERATURE}")
print(f"蒸馏权重Alpha: {ALPHA}")
print(f"学习率: {LEARNING_RATE}")
print("="*60)

🎯 训练配置
实验ID: distill_timeseries_v2
最大轮数: 30
批大小: 64
蒸馏温度: 4.0
蒸馏权重Alpha: 0.7
学习率: 0.0005


## 4. 加载Teacher模型

In [5]:
# 加载Teacher模型
import torch
import pytorch_lightning as pl
from generic_neuromotor_interface.lightning import DiscreteGesturesModule

print("📦 加载Teacher模型...")

# PyTorch 2.6+ weights_only处理
original_torch_load = torch.load
def patched_torch_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return original_torch_load(*args, **kwargs)
torch.load = patched_torch_load

# 加载checkpoint
teacher = DiscreteGesturesModule.load_from_checkpoint(
    TEACHER_CHECKPOINT,
    map_location='cpu'
)

# 恢复torch.load
torch.load = original_torch_load

# 移到GPU并冻结
if torch.cuda.is_available():
    teacher = teacher.cuda()
teacher.eval()
teacher.freeze()

teacher_params = sum(p.numel() for p in teacher.parameters())
print(f"✅ Teacher模型加载完成")
print(f"   参数量: {teacher_params / 1e6:.2f}M")
print(f"   网络配置: stride={teacher.network.stride}, left_context={teacher.network.left_context}")

# 测试输出形状
test_input = torch.randn(2, 16, 16000)
if torch.cuda.is_available():
    test_input = test_input.cuda()
with torch.no_grad():
    test_output = teacher(test_input)
print(f"   输出形状: {test_output.shape}  # [batch, 9, 1598]")

📦 加载Teacher模型...


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'network' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['network'])`.
  rank_zero_warn(


✅ Teacher模型加载完成
   参数量: 6.48M
   网络配置: stride=10, left_context=20
   输出形状: torch.Size([2, 9, 1598])  # [batch, 9, 1598]


## 5. 准备数据模块

In [ ]:
# 准备数据加载器
from generic_neuromotor_interface.data_module import WindowedEmgDataModule
from generic_neuromotor_interface.data import DataSplit
from generic_neuromotor_interface.transforms import DiscreteGesturesTransform
from generic_neuromotor_interface.augmentation import RotationAugmentation

print("📊 准备数据模块...")

# 数据split配置
data_split = DataSplit.from_csv(
    csv_filename=os.path.join(os.path.expanduser(DATA_PATH), f"{TASK}_corpus.csv"),
    pool_test_partitions=True
)

# Transform和Augmentation
transform = DiscreteGesturesTransform(pulse_window=(0, 8))  # (start_offset, end_offset) 40ms at 200Hz
emg_augmentation = RotationAugmentation()

# 创建数据模块
data_module = WindowedEmgDataModule(
    data_location=os.path.expanduser(DATA_PATH),
    data_split=data_split,
    transform=transform,
    emg_augmentation=emg_augmentation,
    window_length=16000,  # 80秒 @ 200Hz
    stride=8000,  # 50% overlap
    batch_size=BATCH_SIZE,
    num_workers=2
)

# Setup数据
data_module.setup('fit')

print(f"✅ 数据模块创建完成")
print(f"   训练batches: {len(data_module.train_dataloader())}")
print(f"   验证batches: {len(data_module.val_dataloader())}")
print(f"   Batch size: {BATCH_SIZE}")

# 验证数据格式
sample_batch = next(iter(data_module.train_dataloader()))
print(f"\n📦 数据格式检查:")
print(f"   EMG shape: {sample_batch['emg'].shape}  # [batch, channels, time]")
print(f"   Targets shape: {sample_batch['targets'].shape}  # [batch, classes, time]")

📊 准备数据模块...


[setup] Loading datasets for split train:   0%|          | 0/80 [00:00<?, ?it/s]

## 6. 定义TimeSeriesStudent模型

**关键改进**: 输出时间序列 `[batch, 9, 1598]` 而非单分类

In [ ]:
# TimeSeriesStudent模型定义
import torch.nn as nn
import torch.nn.functional as F

class TimeSeriesStudent(nn.Module):
    """
    时间序列Student模型 - 输出与Teacher格式一致
    
    架构:
    - Conv1d: 特征提取 + 下采样 (stride=10)
    - GRU: 时间建模 (1层, 128 hidden)
    - Linear: 投影到类别空间
    
    输入: [batch, 16, 16000]
    输出: [batch, 9, 1598]
    参数: ~140K (45x压缩)
    """
    def __init__(self, num_classes=9, input_channels=16, stride=10, left_context=20):
        super().__init__()
        self.num_classes = num_classes
        self.stride = stride
        self.left_context = left_context
        
        # 卷积编码器（模仿Teacher的第一层）
        self.conv1 = nn.Conv1d(input_channels, 128, kernel_size=21, stride=stride)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        # 轻量级时间建模（GRU替代LSTM）
        self.gru = nn.GRU(128, 128, num_layers=1, batch_first=True, dropout=0.0)
        
        # 输出投影
        self.projection = nn.Linear(128, num_classes)
        
    def forward(self, emg):
        """
        前向传播
        
        Args:
            emg: [batch, channels=16, sequence=16000]
        
        Returns:
            logits: [batch, num_classes=9, time_steps=1598]
        """
        # 1. 卷积特征提取 + 下采样
        x = self.conv1(emg)  # [batch, 128, time_steps]
        x = self.relu(self.bn1(x))
        x = self.dropout(x)
        
        # 2. 转置为 [batch, time, features] 用于GRU
        x = x.transpose(1, 2)
        
        # 3. GRU时间建模
        x, _ = self.gru(x)  # [batch, time, 128]
        
        # 4. 投影到类别空间
        x = self.projection(x)  # [batch, time, num_classes]
        
        # 5. 转置回 [batch, num_classes, time]
        x = x.transpose(1, 2)
        
        return x

# 创建Student模型
student = TimeSeriesStudent(
    num_classes=9,
    input_channels=16,
    stride=10,
    left_context=20
)

if torch.cuda.is_available():
    student = student.cuda()

student_params = sum(p.numel() for p in student.parameters())

print("="*60)
print("✅ Student模型创建完成")
print("="*60)
print(f"参数量: {student_params / 1e6:.2f}M")
print(f"压缩比: {teacher_params / student_params:.1f}x")

# 验证输出形状
with torch.no_grad():
    student_output = student(test_input)
    print(f"\n输出形状验证:")
    print(f"  Teacher: {test_output.shape}")
    print(f"  Student: {student_output.shape}")
    print(f"  形状匹配: {test_output.shape == student_output.shape} ✅")
print("="*60)

## 7. 定义蒸馏训练模块

**关键改进**: BCE + MSE蒸馏，使用Teacher的collect_metric评估

In [ ]:
# TimeSeriesDistillationModule定义
from torchmetrics.classification import MulticlassAccuracy

class TimeSeriesDistillationModule(pl.LightningModule):
    """
    时间序列蒸馏训练模块
    
    损失函数:
    - 蒸馏loss: MSE(student_logits, teacher_logits)
    - 真实标签loss: BCEWithLogitsLoss
    - 总loss = alpha * 蒸馏 + (1-alpha) * 真实标签
    
    评估方法:
    - 使用Teacher的collect_metric方法
    - 在事件发生时刻的时间窗口内评估准确度
    """
    def __init__(self, teacher, student, temperature, alpha, learning_rate):
        super().__init__()
        self.teacher = teacher
        self.student = student
        self.temperature = temperature
        self.alpha = alpha
        self.learning_rate = learning_rate
        
        # 损失函数
        self.bce_loss = nn.BCEWithLogitsLoss(reduction='none')
        
        # 评估指标
        self.val_accuracy = MulticlassAccuracy(num_classes=9)
        
        # 保存超参数
        self.save_hyperparameters(ignore=['teacher', 'student'])
        
    def forward(self, x):
        return self.student(x)
    
    def training_step(self, batch, batch_idx):
        emg = batch["emg"]
        targets = batch["targets"]
        
        # 下采样targets以匹配输出长度
        targets = targets[:, :, self.teacher.network.left_context::self.teacher.network.stride]
        
        # Teacher前向传播（不计算梯度）
        with torch.no_grad():
            teacher_logits = self.teacher(emg)
        
        # Student前向传播
        student_logits = self.student(emg)
        
        # 蒸馏损失：MSE on logits
        distill_loss = F.mse_loss(student_logits, teacher_logits)
        
        # 真实标签损失：BCE
        student_loss = self.bce_loss(student_logits, targets).mean()
        
        # 总损失
        loss = self.alpha * distill_loss + (1 - self.alpha) * student_loss
        
        # 记录指标
        self.log('train/loss', loss, prog_bar=True, on_step=True, on_epoch=True)
        self.log('train/distill_loss', distill_loss, on_step=False, on_epoch=True)
        self.log('train/student_loss', student_loss, on_step=False, on_epoch=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        emg = batch["emg"]
        targets = batch["targets"]
        
        # 下采样targets
        targets = targets[:, :, self.teacher.network.left_context::self.teacher.network.stride]
        
        # Student推理
        logits = self.student(emg)
        
        # BCE loss
        loss = self.bce_loss(logits, targets).mean()
        
        # 使用collect_metric计算准确度
        self.collect_metric(logits, targets, phase='val')
        
        self.log('val/loss', loss, prog_bar=True, on_epoch=True, on_step=False)
        
        return loss
    
    def collect_metric(self, logits, target, phase):
        """
        复制Teacher的collect_metric方法
        在事件发生时刻的时间窗口内评估分类准确度
        """
        device = logits.device
        w_start = 10  # 50 ms at 200 Hz
        w_end = 30  # 150 ms at 200 Hz
        
        probs = torch.sigmoid(logits)
        
        # 转置为 [batch, time, classes]
        probs = probs.transpose(1, 2)
        target = target.transpose(1, 2)
        
        y = target.to(torch.int32)
        y_class = []
        y_hat_class = []
        
        for batch in range(y.shape[0]):
            y_diff = torch.diff(y[batch], axis=0)
            indices = torch.argwhere(y_diff == 1)
            
            for index in indices:
                start = max(index[0] - w_start, 0)
                end = min(index[0] + w_end, y.shape[1])
                
                y_hat = probs[batch, start:end, :]
                flattened_index = y_hat.argmax()
                _, cols = y_hat.shape
                col = flattened_index % cols
                
                y_hat_class.append(col)
                y_class.append(index[1])
        
        if len(y_class) > 0:
            y_class = torch.stack(y_class).long().to(device)
            y_hat_class = torch.stack(y_hat_class).long().to(device)
        else:
            y_class = torch.zeros(1, dtype=torch.int64, device=device)
            y_hat_class = torch.zeros(1, dtype=torch.int64, device=device)
        
        self.val_accuracy.update(y_hat_class, y_class)
        self.log(f'{phase}/accuracy', self.val_accuracy, on_step=False, on_epoch=True, prog_bar=True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.student.parameters(),
            lr=self.learning_rate,
            weight_decay=1e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=MAX_EPOCHS
        )
        return [optimizer], [scheduler]

# 创建蒸馏模型
distill_model = TimeSeriesDistillationModule(
    teacher=teacher,
    student=student,
    temperature=TEMPERATURE,
    alpha=ALPHA,
    learning_rate=LEARNING_RATE
)

print("✅ 蒸馏训练模块创建完成")
print(f"   Temperature: {TEMPERATURE}")
print(f"   Alpha (蒸馏权重): {ALPHA}")
print(f"   Learning Rate: {LEARNING_RATE}")

## 8. 配置训练器和Callbacks

In [ ]:
# 配置训练器
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

# 日志目录
log_dir = os.path.join(SAVE_DIR, EXPERIMENT_ID)
os.makedirs(log_dir, exist_ok=True)

# Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join(log_dir, 'checkpoints'),
    filename='epoch={epoch:02d}-val_acc={val/accuracy:.4f}',
    monitor='val/accuracy',
    mode='max',
    save_top_k=3,
    save_last=True,
    verbose=True
)

early_stop_callback = EarlyStopping(
    monitor='val/accuracy',
    patience=10,
    mode='max',
    verbose=True
)

lr_monitor = LearningRateMonitor(logging_interval='epoch')

# Logger
logger = TensorBoardLogger(
    save_dir=log_dir,
    name='tensorboard_logs'
)

# Trainer
trainer = pl.Trainer(
    max_epochs=MAX_EPOCHS,
    accelerator='auto',
    devices=1,
    callbacks=[checkpoint_callback, early_stop_callback, lr_monitor],
    logger=logger,
    precision=16,  # 混合精度训练
    gradient_clip_val=1.0,
    log_every_n_steps=10,
    enable_progress_bar=True
)

print("="*60)
print("✅ 训练器配置完成")
print("="*60)
print(f"日志目录: {log_dir}")
print(f"最大轮数: {MAX_EPOCHS}")
print(f"早停patience: 10")
print(f"混合精度: FP16")
print("="*60)

## 9. 执行训练 🚀

In [ ]:
# 开始训练
import time

print("="*60)
print("🚀 开始蒸馏训练")
print("="*60)
print(f"Teacher准确度: 47.42% (validation)")
print(f"目标Student准确度: 35-45%")
print(f"预计训练时间: ~1-1.5小时 (T4 GPU)")
print("="*60)

start_time = time.time()

# 训练
trainer.fit(distill_model, data_module)

elapsed_time = time.time() - start_time
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)

print("\n" + "="*60)
print("✅ 训练完成！")
print("="*60)
print(f"训练时间: {hours}小时 {minutes}分钟")
print(f"最佳checkpoint: {checkpoint_callback.best_model_path}")
print(f"最佳验证准确度: {checkpoint_callback.best_model_score:.4f}")
print("="*60)

## 10. 结果评估与对比

In [ ]:
# 加载最佳模型
print("📊 评估最佳模型...\n")

best_model = TimeSeriesDistillationModule.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    teacher=teacher,
    student=student,
    temperature=TEMPERATURE,
    alpha=ALPHA,
    learning_rate=LEARNING_RATE
)

# 在验证集上评估
val_results = trainer.validate(best_model, data_module, verbose=False)

print("="*60)
print("📊 最终结果对比")
print("="*60)
print(f"Teacher模型:")
print(f"  参数量: {teacher_params / 1e6:.2f}M")
print(f"  验证准确度: 47.42%")
print()
print(f"Student模型 (TimeSeriesStudent):")
print(f"  参数量: {student_params / 1e6:.2f}M")
print(f"  验证准确度: {val_results[0]['val/accuracy']*100:.2f}%")
print(f"  验证损失: {val_results[0]['val/loss']:.4f}")
print()
print(f"压缩比: {teacher_params / student_params:.1f}x")
print(f"准确度保留: {val_results[0]['val/accuracy'] / 0.4742 * 100:.1f}%")
print("="*60)

# 保存关键信息
result_summary = {
    'experiment_id': EXPERIMENT_ID,
    'teacher_params': teacher_params,
    'student_params': student_params,
    'compression_ratio': teacher_params / student_params,
    'teacher_val_acc': 0.4742,
    'student_val_acc': val_results[0]['val/accuracy'].item(),
    'student_val_loss': val_results[0]['val/loss'].item(),
    'best_checkpoint': checkpoint_callback.best_model_path,
    'training_time_minutes': int(elapsed_time / 60)
}

import json
with open(os.path.join(log_dir, 'result_summary.json'), 'w') as f:
    json.dump(result_summary, f, indent=2)

print(f"\n✅ 结果已保存到: {log_dir}/result_summary.json")

## 11. 导出Student模型（可选）

In [ ]:
# 导出纯Student模型（不含Training wrapper）
student_save_path = os.path.join(log_dir, 'student_model.pt')
torch.save({
    'model_state_dict': best_model.student.state_dict(),
    'model_config': {
        'num_classes': 9,
        'input_channels': 16,
        'stride': 10,
        'left_context': 20
    },
    'val_accuracy': val_results[0]['val/accuracy'].item(),
    'parameters': student_params
}, student_save_path)

print(f"✅ Student模型已导出到: {student_save_path}")
print(f"\n💡 下一步:")
print(f"   1. 量化模型: 使用PyTorch的量化工具")
print(f"   2. 转换为TFLite: 用于ESP32部署")
print(f"   3. 继续训练: 加载checkpoint训练更多epochs")

## 总结

### ✅ 完成的工作
1. **正确的任务定义**: 时间序列事件检测，输出 `[batch, 9, 1598]`
2. **正确的模型架构**: TimeSeriesStudent (140K参数, 45x压缩)
3. **正确的蒸馏方法**: BCE + MSE蒸馏，保留时间序列信息
4. **正确的评估方法**: 使用Teacher的collect_metric

### 📊 预期vs实际
- **预期准确度**: 35-45%
- **实际准确度**: (训练后查看)
- **Teacher准确度**: 47.42%

### 🎯 下一步优化方向
1. **增加训练轮数**: 50-100 epochs
2. **调整超参数**: temperature (2-6), alpha (0.5-0.9)
3. **模型架构**: 增加GRU层数或hidden size
4. **量化部署**: INT8量化 → TFLite → ESP32